In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [2]:
import mlrose
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import process_time

In [5]:
print("Running Travel Salesman...")

distances = [(0, 1, 0.274), (0, 2, 1.367), (1, 2, 1.091), (0, 3, 1.422), (1, 3, 1.153), (2, 3, 1.038),
                        (0, 4, 1.870), (1, 4, 1.602), (2, 4, 1.495), (3, 4, 0.475), (0, 5, 1.652), (1, 5, 1.381),
                        (2, 5, 1.537), (3, 5, 0.515), (4, 5, 0.539), (0, 6, 1.504), (1, 6, 1.324), (2, 6, 1.862),
                        (3, 6, 1.060), (4, 6, 1.097), (5, 6, 0.664), (0, 7, 1.301), (1, 7, 1.031), (2, 7, 1.712),
                        (3, 7, 1.031), (4, 7, 1.261), (5, 7, 0.893), (6, 7, 0.350), (0, 8, 1.219), (1, 8, 0.948),
                        (2, 8, 1.923), (3, 8, 1.484), (4, 8, 1.723), (5, 8, 1.396), (6, 8, 0.872), (7, 8, 0.526),
                        (0, 9, 0.529), (1, 9, 0.258), (2, 9, 1.233), (3, 9, 1.137), (4, 9, 1.560), (5, 9, 1.343),
                        (6, 9, 1.131), (7, 9, 0.816), (8, 9, 0.704)]

# Define Travel Salesman objective function and problem
fitness = mlrose.TravellingSales(distances=distances)
problem = mlrose.TSPOpt(length=10, fitness_fn=fitness, maximize=False)


RANDOM_SEED = 42
MAX_ATTEMPTS = 100

#%% tuning for SA
curve_list = []
decays = [0.999, 0.99]
for d in decays:
    schedule = mlrose.GeomDecay(decay=d)
    _, _, curve = mlrose.simulated_annealing(
        problem,
        schedule=schedule,
        max_attempts=MAX_ATTEMPTS,
        max_iters=500,
        curve=True,
        random_state=RANDOM_SEED,
    )
    curve_list.append(curve)
    
df = pd.DataFrame(curve_list).transpose()
df.columns = decays
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("TravelSalesman: Fitness curve vs decay rate in SA")
plt.savefig("output/travelsalesman_sa_decay.png")
plt.close()

print(df.max())

Running Travel Salesman...
0.999   -6.19
0.990   -6.19
dtype: float64


In [6]:
#%% tuning for GA
curve_list = []
pop_sizes = [50, 100, 200, 400]
for p in pop_sizes:
    _, _, curve = mlrose.genetic_alg(
        problem,
        max_attempts=MAX_ATTEMPTS,
        max_iters=500,
        pop_size=p,
        curve=True,
        random_state=RANDOM_SEED,
    )
    curve_list.append(curve)

df = pd.DataFrame(curve_list).transpose()
df.columns = pop_sizes
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("Travel Salesman: Fitness curve vs population size in GA")
plt.savefig("output/travelsalesman_ga_pop.png")
plt.close()

print(df.max())

50    -8.009
100   -7.697
200   -6.663
400   -7.151
dtype: float64


In [8]:
#%% tuning for MIMIC
curve_list = []
nth_pct = [0.2, 0.4]
for p in nth_pct:
    _, _, curve = mlrose.mimic(
        problem,
        max_attempts=MAX_ATTEMPTS,
        max_iters=50,
        keep_pct=p,
        curve=True,
        random_state=RANDOM_SEED,
    )
    curve_list.append(curve)

df = pd.DataFrame(curve_list).transpose()
df.columns = nth_pct
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("Travel Salesman: Fitness curve vs nth percentile in MIMIC")
plt.savefig("output/travelsalesman_mimic_nth.png")
plt.close()

print(df.max())



0.2   -6.829
0.4   -7.122
dtype: float64


In [7]:
#%% Putting together
curve_list = []
time_list = []
n_eval = []
algo_list = ["RHC", "SA", "GA", "MIMIC"]

# RHC
t1 = process_time()
_, _, curve = mlrose.random_hill_climb(
    problem,
    max_attempts=MAX_ATTEMPTS,
    max_iters=500,
    curve=True,
    random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append(np.argmax(curve) + 1)

# SA
t1 = process_time()
_, _, curve = mlrose.simulated_annealing(
    problem,
    max_attempts=MAX_ATTEMPTS,
    max_iters=500,
    curve=True,
    random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append(np.argmax(curve) + 1)

# GA
t1 = process_time()
_, _, curve = mlrose.genetic_alg(
    problem, max_attempts=MAX_ATTEMPTS, curve=True, random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append((np.argmax(curve) + 1) * 200)

# MIMIC
t1 = process_time()
_, _, curve = mlrose.mimic(
    problem,
    max_attempts=MAX_ATTEMPTS,
    keep_pct=0.4,
    curve=True,
    random_state=RANDOM_SEED,
)
t2 = process_time()
time_list.append((t2 - t1) / len(curve))
curve_list.append(curve)
n_eval.append((np.argmax(curve) + 1) * 200)

df = pd.DataFrame(curve_list).transpose()
df.columns = algo_list
df.plot()
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("Travel Salesman: Fitness curve vs algorithms")
plt.savefig("output/travelsalesman_algo.png")
plt.close()

print("time per iteration:")
print(time_list)
print("number of func eval reaching maxima:")
print(n_eval)
print("maxima reached:")
print(df.max())

time per iteration:
[0.0002118508287292601, 0.0001874039999999937, 0.050935648000000014, 0.3101861764705882]
number of func eval reaching maxima:
[81, 261, 5000, 400]
maxima reached:
RHC     -6.190
SA      -6.190
GA      -6.663
MIMIC   -7.122
dtype: float64
